In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# EDA Question 1
# How does the 'Variance Ratio' (how much the actual cost varies compared to the original budget) of completed projects vary by province, and are there regions where cost overruns are consistently higher than the national average? 

df_master = pd.read_csv("./data/merged/typhoon-info-infra-project.csv")

# 1. Isolate the "Final State" for each province
# We take the .max() of the cumulative columns because they represent 
# the total accumulation up to the most recent data point.
province_final_stats = df_master.groupby(['Province', 'Region']).agg({
    'Cumulative_Budget_To_Date': 'max',
    'Cumulative_Variance_To_Date': 'max'
}).reset_index()

# 2. Calculate the Final Variance Ratio for the province
# This represents the total percentage of 'over-spending' for that province's history
province_final_stats['Final_Variance_Ratio'] = (
    province_final_stats['Cumulative_Variance_To_Date'] / 
    province_final_stats['Cumulative_Budget_To_Date']
)

# 3. Calculate the National Baseline using these FINAL province ratios
# This ensures every province contributes equally to the average, regardless of typhoon frequency
national_avg_ratio = province_final_stats['Final_Variance_Ratio'].mean()

print(f"Corrected National Average Variance Ratio: {national_avg_ratio:.2%}")
print(province_final_stats.sort_values(by='Final_Variance_Ratio', ascending=False).head())

plt.figure(figsize=(14, 8))

# Sorting to make outliers (like Sorsogon) obvious
province_final_stats = province_final_stats.sort_values('Final_Variance_Ratio', ascending=False)

sns.barplot(
    data=province_final_stats, 
    x='Province', 
    y='Final_Variance_Ratio', 
    hue='Region', 
    dodge=False
)

# The correct National Baseline
plt.axhline(national_avg_ratio, color='red', linestyle='--', label=f'National Avg ({national_avg_ratio:.2%})')

plt.title('Final Aggregate Variance Ratio per Province (Corrected)')
plt.ylabel('Aggregate Variance Ratio (Total Overrun %)')
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.savefig('data/eda-outputs/province_variance_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Justification of Visualization:

# We utilized a sorted bar chart with a red horizontal reference line representing the national average (1.20%). A bar chart is the most effective way to compare a specific metric across discrete categories (provinces), while the reference line allows for immediate identification of outliers.

# Interpretation & Insights:

# Efficiency Outliers: Sorsogon and Batanes demonstrate the highest fiscal friction, with Sorsogon's overrun ratio being nearly eight times higher than the national average.

# Regional Pattern: There is a noticeable clustering of Region V (Bicol) provinces at the high end of the spectrum, suggesting that management challenges may be regional rather than isolated to single provinces.

# Conclusion on EDA question 1: This randomness in efficiency suggests that "fiscal friction" is a systemic issue across the Philippines, not necessarily tied to a province's weather risk, as even "dry" provinces show significant variance.